## Setup environments

In [1]:
import os

from openai import OpenAI
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())

client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

llm_model = 'gpt-4o-mini'
chat_model = ChatOpenAI(temperature=0.0, model=llm_model)

## Simple Chain

In [ ]:
prompt = PromptTemplate(
    input_variables=["language"],
    template="How do you say good morning in {language}?"
)

# chain = LLMChain(llm=chat_model, prompt=prompt)
chain = prompt | chat_model
response = chain.invoke(input={"language": "Korean"}).content
print(response)

In Korean, "good morning" is said as "안녕하세요" (annyeonghaseyo). This is a common greeting used throughout the day, but it can also be used specifically in the morning. If you want to be more formal, you can say "안녕하십니까" (annyeonghasimnikka).


## Simple Sequential Chain

In [ ]:
from langchain.chains import LLMChain, SequentialChain

story_template = """ 
동화작가로서 지역 {location}과 주인공 이름 {name}을 이용해서
간단하고 짧은 (90개 단어 정도) 자장가를 하나 작성해줘.
"""
prompt_story = PromptTemplate(input_variables=["location", "name"], template=story_template)
chain_story = LLMChain(prompt=prompt_story, llm=chat_model, output_key = "story")

In [41]:
story = chain_story.invoke(input={"location": "포항", "name": "홍길동"})
print(story['story'])

포항의 바다, 달빛 아래  
홍길동이 잠이 들어요.  
파도 소리, 부드러운 노래,  
꿈속에서 모험을 해요.  

별빛이 반짝, 하늘에 춤추고,  
홍길동의 미소가 빛나요.  
바람이 속삭여, "잘 자요, 친구!"  
포항의 밤, 평화롭게 흐르죠.  

꿈나라로 떠나는 길,  
사랑과 행복 가득해요.  
홍길동아, 편안히 자렴,  
내일은 또 다른 날이야.


In [ ]:
translate_template = """
다음과 같은 자장가 {story}를 언어 {language}로 번역해줘.
번역할 때 어린이들이 좋아할 만한 단어들을 사용해야 하는 것을 잊지 말구.

번역:
"""
prompt_translate = PromptTemplate(input_variables=["story", "language"], template=translate_template)
chain_translate = LLMChain(prompt=prompt_translate, llm=chat_model, output_key="translated")

In [42]:
chain_translate.invoke(input={"story": story, "language": "영어"})["translated"]

'Here’s a gentle lullaby translated into English, using words that children would enjoy:\n\n---\n\n{\'location\': \'Pohang\', \'name\': \'Hong Gil-dong\', \'story\': \'In the sea of Pohang, under the moonlight,  \\nHong Gil-dong drifts off to sleep.  \\nThe sound of waves, a soft, sweet song,  \\nIn his dreams, adventures go on and on.  \\n\\nStars are twinkling, dancing in the sky,  \\nHong Gil-dong’s smile shines bright and high.  \\nThe wind whispers softly, "Goodnight, my friend!"  \\nIn the night of Pohang, peace flows without end.  \\n\\nOn the path to dreamland,  \\nFilled with love and happiness so grand.  \\nSleep tight, dear Hong Gil-dong,  \\nTomorrow brings another day, bright and strong.\'}'

In [43]:
overall_chain = SequentialChain(
    chains=[chain_story, chain_translate],
    input_variables=["location", "name", "language"],  # Initial inputs
    output_variables=["translated"]  # Final output expected from SequentialChain
)

response = overall_chain.invoke(input={
    "location": "포항",
    "name": "홍길동",
    "language": "영어"
})

print(response['translated'])

Here’s a gentle lullaby translation that uses child-friendly language:

---

In Pohang's sea, under the moonlight,  
Hong Gil-dong drifts off to dreamland bright.  
Waves are singing, the wind's a tune,  
Stars are twinkling, wrapping hearts like a cocoon.  

Little fish are dancing, oh so free,  
Clouds in the sky join in harmony.  
Close your eyes, dear Hong Gil-dong,  
Dream sweet dreams, where you belong.  

Pohang's night, a peaceful time,  
Filled with love, oh so sublime.  
Snuggled in Mommy's warm embrace,  
Dream happy dreams, in this cozy place.  

--- 

I hope this captures the essence of the original while being appealing to children!
